In [1]:
from Alpaca_keyes import API_KEY, SECRET_KEY
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo

from alpaca.data.timeframe import TimeFrame, TimeFrameUnit
from alpaca.data.historical.stock import StockHistoricalDataClient
from alpaca.data.requests import StockBarsRequest

from Indicators import EMA, ATR

In [2]:
api_key = API_KEY
secret_key = SECRET_KEY

symbol = "TSLA"

stock_historical_data_client = StockHistoricalDataClient(api_key, secret_key)

In [3]:
now = datetime.now(ZoneInfo("America/New_York"))

req = StockBarsRequest(
    symbol_or_symbols = [symbol],
    timeframe=TimeFrame(amount = 15, unit = TimeFrameUnit.Minute), # specify timeframe
    start = now - timedelta(days = 51),                          # specify start datetime, default=the beginning of the current day.
    # end_date=None,                                        # specify end datetime, default=now
    #limit = 2,                                               # specify limit
)

In [4]:
# timestamp is in UTC time
df = stock_historical_data_client.get_stock_bars(req).df

In [5]:
df = df.reset_index()
#df['symbol'] = symbol

#dataF = EMA(df).EMA_50(50)
#dataF = ATR(dataF).calculate_chandelier_exit()

In [ ]:
display(df)

In [12]:
import plotly.graph_objects as go
from Brokers.Alpaca.Testing.EasyBot import ColumnNames

In [ ]:
fig2 = go.Figure(data=[go.Candlestick(x=dataF.index, # dataF["Datetime"]
                open=dataF[ColumnNames.OPEN],
                high=dataF[ColumnNames.HIGH],
                low=dataF[ColumnNames.LOW],
                close=dataF[ColumnNames.CLOSE])])

fig2.add_trace(go.Scatter(
    x= dataF.index, # dataF["Datetime"]
    y=dataF[ColumnNames.EMA_50],
    mode='lines',
    name='50-day EMA',
    line=dict(color='blue', width=2)
))

fig2.update_layout(height=800)

fig2.show()

### Get live data

In [14]:
import nest_asyncio
nest_asyncio.apply()

import asyncio
from alpaca.data.live.stock import StockDataStream
import pandas as pd

In [15]:
stock_data_stream_client = StockDataStream(api_key, secret_key)
data_stream_list = []

In [16]:
async def stock_data_stream_handler(bar):
    data = {
        'timestamp': [bar.timestamp],
        'open': [bar.open],
        'high': [bar.high],
        'low': [bar.low],
        'close': [bar.close],
        'volume': [bar.volume]
    }

    data_stream_list.append(data)

In [17]:
"""async def stock_data_stream_handler(data):
    
    print(data)
    print(type(data))
    print(data.df)"""

symbols = [symbol]

stock_data_stream_client.subscribe_bars(stock_data_stream_handler, *symbols)

In [ ]:
stock_data_stream_client.run()

In [ ]:
df = pd.DataFrame(data_stream_list)

#### Testing

In [ ]:
import threading
import time

def consumer_thread():
    try:
        # make sure we have an event loop, if not create a new one
        loop = asyncio.get_event_loop()
        loop.set_debug(True)
    except RuntimeError:
        asyncio.set_event_loop(asyncio.new_event_loop())

    global data_stream
    data_stream = StockDataStream(api_key, secret_key)
    
    async def stock_data_stream_handler(data):
        print(data)
        print(type(data))

    data_stream.subscribe_bars(stock_data_stream_handler, *symbols)

    data_stream.run()

def additional_function():
    print("Running additional function")
    # Perform tasks here
    time.sleep(5)  # Simulate a task taking some time
    print("Additional function completed")

if __name__ == '__main__':
    loop = asyncio.get_event_loop()

    while 1:
        threading.Thread(target=consumer_thread).start()
        time.sleep(5)
        loop.run_until_complete(data_stream.stop_ws())

        # Run the additional function in a separate thread
        additional_thread = threading.Thread(target=additional_function)
        additional_thread.start()
        
        # Wait for the additional function to complete
        additional_thread.join()

        time.sleep(20)


Now I can store the data in a pandas dataframe

In [ ]:
stock_data_stream_client = StockDataStream(api_key, secret_key)

async def stock_data_stream_handler(data):
    print(data)
    print(type(data))

symbols = [symbol]

stock_data_stream_client.subscribe_quotes(stock_data_stream_handler, *symbols)
stock_data_stream_client.subscribe_trades(stock_data_stream_handler, *symbols)

stock_data_stream_client.run()